# To do

- [X] write usable json files
- [X] allocate all files in folder
- [X] print multiple poi on map
- [ ] connect to open folder/files
- [ ] make it runable - GUI Win/Html....

# Sources

[Source 1](https://nbviewer.jupyter.org/github/gotoariel/folium-demo/blob/master/demo/demo.ipynb)

[Source 2](http://blog.schockwellenreiter.de/2020/05/2020051601.html)

# Import libraries

In [33]:
import json, string

#Folium uses the Leaflet API to allow users to write Python code to generate and manipulate interactive JavaScript maps. This also allows for drawing those maps in Jupyter notebooks
import folium
#import requests
import geocoder
#import pandas as pd
import matplotlib
import numpy as np
import pandas as pd
import os
import webbrowser
from folium import Map, Marker#, GeoJson, LayerControl
#from ediblepickle import checkpoint


%matplotlib inline 

# 1.0 Write location based files

In [40]:
# for testing purpose generated json files

In [35]:
data = {}
#data = []
data = {
    'name': "Regenbecken-4",
    'type': 'Speicher',
    'adress': 'Auggen, 79424, Germany',
    'desc': "Becken",
    'year_build': 1700,
    'location': '47.790770, 7.583286'
}

with open('location.txt', 'w') as outfile:
    json.dump(data, outfile)

# 2.0 Import data from dir

In [34]:
path = 'data/'

In [35]:
file_dict = []
path_list = []

# function to walk through dirs and collect relevant documents "location.txt"

def file_reader(path):
    for root, dirs, files in os.walk(path): # searches in folders, subfolders and files
        for name in dirs:
            file_path = os.path.join(root, name)
            for txt_file in os.listdir(file_path):
            
                if txt_file == 'location.txt':
                    #folder = os.path(file_path)
                    file = os.path.join(file_path, txt_file) # get filepath for each file
                    path_list.append(file)
    
                    with open(file) as json_file: # collect json contend
                        data = json.load(json_file)
                        file_dict.append(data) # append json dict to file_dict
                        #file_dict.append(folder)


In [40]:
file_reader(path)

# Mapping

In [36]:
# get my loction
my_loc = geocoder.ip('me')

In [41]:
#locations = pd.Series(file_dict)
#location_base  = locations[0]['location'].split(",")

locations_map = Map(location = [my_loc.lat, my_loc.lng],
                    height= '100',
                    tiles = 'OpenStreetMap',
                    zoom_start = 8)
locations_map.add_child(
   Marker(location = [my_loc.lat, my_loc.lng],
                      tooltip = "Mein Standort",
                      icon = folium.Icon(color = 'blue')));

In [42]:
for i in pd.Series(file_dict):
    name = i['name']
    address = i['adress']
    description = i['desc']
    locations = i['location'].split(",")
    year_build = i['year_build']
    folium.features.RegularPolygonMarker(location = [locations[0], locations[1]],
                                        tooltip = name,
                                        popup = "Name: " + name + "\nTyp: " + description + "\nAdresse: " + address,
                                        #zoom = 200,
                                        #number_of_sides = 3,
                                        #radius = 5,
                                        #weight = 1,
                                        #fill_opacity = 0.8
                                        ).add_to(locations_map)
    
locations_map

# 4.0 Add Href to open content

In [43]:
for i in pd.Series(file_dict):
    name = i['name']
    address = i['adress']
    #path = "/home/andres/DataScience/Archive-Mapping/data/datassss 2/location.txt"
    description = i['desc']
    locations = i['location'].split(",")
    year_build = i['year_build']

    folium.features.RegularPolygonMarker(location = [locations[0], locations[1]],
                                        tooltip = name,
                                        #popup = ('<a href="/home/andres/DataScience/Archive-Mapping/data/test 6">open dir</a>'), #"Name: " + name + "\nTyp: " + description + "\nAdresse: " + address,
                                        #zoom = 200,
                                        #number_of_sides = 3,
                                        #radius = 5,
                                        #weight = 1,
                                        #fill_opacity = 0.8
                                        ).add_to(locations_map)
    
locations_map.save("locations_map.html")
webbrowser.open("locations_map.html", new=2); #open new tab in webbrowser
    
#locations_map

# 5.0 unse GUI

In [ ]:
import kivy

